In [ ]:
# import sys
# !"{sys.executable}" -m pip install mido

In [ ]:
import mido
import os
import torch
import torch.nn as nn
import time
from torch import optim
import torch.nn.functional as F
from numpy import random
from mido import MidiTrack
from mido import MidiFile, MetaMessage
st = 0.0013354687499999999 * 3
SOS_token = 0
hid_size = 512
time_cl = [0, 1, 2, 3, 5, 7, 11, 15, 20, 23, 35, 80, 122, 187]
vel_cl = [0, 15, 33, 64, 80, 104, 125]
tempo = 2000000
note_cl = []

In [ ]:
def encoding(events):
    ans = []
#     print(events)
    for event in events:
        if (event[0] == 'v'):
            ans.append(event[1])
        if (event[0] == 't'):
            ans.append(len(vel_cl) + event[1])
        if (event[0] == 'no'):
            ans.append(len(vel_cl) + len(time_cl) + event[1])
        if (event[0] == 'nof'):
            ans.append(len(vel_cl) + len(time_cl) + len(note_cl) + event[1])
    return ans

def decoding(ans):
    events = []
    for n in ans:
#         print(ans)
        if (n < len(vel_cl)):
            events.append(['v', n])
        elif (n < len(vel_cl) + len(time_cl)):
            events.append(['t', n - len(vel_cl)])
        elif (n < len(vel_cl) + len(time_cl) + len(note_cl)):
            events.append(['no', n - len(vel_cl) - len(time_cl)])
        else:
            events.append(['nof', n - len(vel_cl) - len(time_cl) - len(note_cl)])
    return events

In [ ]:
count = 0
velocityes = {}
times = {}
notes = {}
def preprocess(path):
    global count
    global times, velcityes, notes, note_cl
    mid = MidiFile('midi/'+path)    
    messages = []
    time = 0
    vel = 0

    for msg in mid:
        if (msg.type == 'note_off' or msg.type == 'note_on'):
            msg.time = int(msg.time/st)
#             if (1):
#                 if(msg.velocity in velocityes):
#                     velocityes[msg.velocity] += 1
#                 else:
#                     velocityes[msg.velocity] = 1
#                 if(msg.time in times):
#                     times[msg.time] += 1
#                 else:
#                     times[msg.time] = 1
#                 if(msg.note in notes):
#                     notes[msg.note] += 1
#                 else:
#                     notes[msg.note] = 1
            messages.append(msg)
#             time = max(time, msg.time)
#             vel = max(vel, msg.velocity)
    for msg in messages:
        if (msg.note not in note_cl):
            note_cl.append(msg.note)
        msg.note = note_cl.index(msg.note)
        vel = msg.velocity
        new_vel = 0
        for i in range(len(vel_cl)):
            if (abs(vel_cl[i] - vel) < abs(vel_cl[new_vel] - vel)):
                new_vel = i
        msg.velocity = new_vel
    
        time = msg.time
        new_time = 0
        for i in range(len(time_cl)):
            if (abs(time_cl[i] - time) < abs(time_cl[new_time] - time)):
                new_time = i
        msg.time = new_time
        
    events = []    
    last_vel = -1
    last_time = -1
    for msg in messages:
        if (msg.velocity != last_vel):
            last_vel = msg.velocity
            events.append(['v', msg.velocity])
        if (msg.time != last_time):
            last_time = msg.time
            events.append(['t', msg.time])
        if (msg.type == 'note_on'):
            events.append(['no', msg.note])
        if (msg.type == 'note_off'):
            events.append(['nof', msg.note])
    
    v = encoding(events)
    return v

def postprocess(v):
    global count
    d = decoding(v)
    events = d
    
    
    for i in range(len(events)):
        event = events[i]
        if (event[0] == 'v'):
            event[1] = vel_cl[event[1]]
        elif (event[0] == 't'):
            event[1] = time_cl[event[1]]
        else:
            event[1] = note_cl[event[1]]
    track = MidiTrack()
    track.append(MetaMessage("set_tempo", tempo=tempo, time=0))
    velocity_n = 0
    time_n = 0
    for i in range(len(events)):
        if (events[i][0] == 'v'):
            velocity_n = events[i][1]
        if (events[i][0] == 't'):
            time_n = events[i][1]
        if (events[i][0] == 'no'):
            track.append(mido.Message('note_on', note=events[i][1], time=time_n, velocity=velocity_n))
        if (events[i][0] == 'nof'):
            track.append(mido.Message('note_off', note=events[i][1], time=time_n, velocity=velocity_n))
    mid_f = MidiFile()
    mid_f.tracks.append(track)
    mid_f.save('midi_pr/new_song' + str(count) +'.mid')
    count+=1

In [ ]:
times = {}
notes = {}
count = 0
songs = []
velocityes = {}
for path in os.listdir('midi'):
    songs.append(preprocess(path))
# for t in sorted(times):
#     print(t, times[t])
# for t in sorted(velocityes):
#     print(t, velocityes[t])
# for t in sorted(notes):
#     if(notes[t] > 100):
#         print(t, notes[t])

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def train(target_tensor, decoder, decoder_optimizer, criterion, decoder_hidden):
    decoder_optimizer.zero_grad()

    target_length = target_tensor.size(2)//20
    start = target_tensor.size(2) // 3
    
    loss = 0

    decoder_input = torch.tensor([[SOS_token]])
    print(target_length)
    for di in range(start, target_length + start):
#         print(di)
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_tensor[0,:,di])
        decoder_input = target_tensor[0,:,di]
#         print(decoder_input)
#         print(decoder_output.topk(1))

    loss.backward()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
def trainIters(decoder, decoder_optimizer, n_iters, songs_hid, print_every=1000):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        number = iter % len(songs)
        target_tensor = torch.LongTensor([songs[number - 1]]).view(1,1,-1)
        
        loss = train(target_tensor,
                     decoder, decoder_optimizer, criterion, songs_hid[number])
#         print("ok")
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

In [ ]:
# songs_hid = []
# SOS_token = 2*len(note_cl) + len(time_cl) + len(vel_cl)
# for i in range(len(songs)):
#     songs_hid.append(torch.FloatTensor([random.random() for _ in range(hid_size)]).view(1,1,hid_size))
# classes = SOS_token + 1
# decoder = DecoderRNN(hid_size, classes)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.0001)

In [ ]:
trainIters(decoder, decoder_optimizer,  100000, songs_hid, 2)

In [ ]:
def evaluate(decoder, decoder_hidden):
    with torch.no_grad():
        decoder_optimizer.zero_grad()

        target_length = 1000
        outputs = []
        decoder_input = torch.tensor([[SOS_token]])
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            outputs.append(topi[0].item())
            decoder_input = torch.tensor([[topi[0].item()]])
        return outputs


decoder_hidden = torch.FloatTensor([random.random() for _ in range(hid_size)]).view(1,1,-1)
print(evaluate(decoder, decoder_hidden))
postprocess(evaluate(decoder, decoder_hidden))

In [ ]:
print(len(songs[0]))

In [ ]:
path = os.getcwd() + '\models\model1'
torch.save({
                    'model_state_dict': decoder.state_dict(),
                    }, path)
# print(path)